## <center>This Model will Attempt to Predict the Type of Bicycle Used from Trip Data</center>

In [56]:
# Eliminating Warnings
import warnings
warnings.filterwarnings('ignore')

In [57]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

In [58]:
# Set up data path
data_path = 'data'

In [59]:
# Load base data to dataframe
all_data_file = 'all_data.csv'
all_df = pd.read_csv(os.path.join(data_path, all_data_file))
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,11/28/16,Citibike from Work,0.81,NaN,7.0,0.12,6.94,Citibike
1,10/15/19,Wards Island,14.20,1.0,44.0,1.73,8.19,Specialized
2,7/28/19,Roosevelt Island,4.25,NaN,39.0,0.65,6.54,Specialized
3,6/8/20,From Warehouse Wines,2.83,NaN,31.0,0.52,5.48,Citibike
4,9/6/20,Dn ES UP WS,17.30,1.0,55.0,1.92,9.03,Specialized
...,...,...,...,...,...,...,...,...
349,9/30/19,CP BL x 2,13.60,1.0,44.0,1.73,7.85,Specialized
350,11/19/16,Down ES UP WS,16.10,1.0,56.0,1.93,8.33,Specialized
351,6/7/19,CP BL,7.47,NaN,54.0,0.90,8.30,Specialized
352,6/8/20,To Despana,3.64,NaN,35.0,0.58,6.24,Citibike


In [60]:
# Change NaN values to 0 - only occurs on hours column so this is accurate
all_df['Hours'] = all_df['Hours'].fillna(0)
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,11/28/16,Citibike from Work,0.81,0.0,7.0,0.12,6.94,Citibike
1,10/15/19,Wards Island,14.20,1.0,44.0,1.73,8.19,Specialized
2,7/28/19,Roosevelt Island,4.25,0.0,39.0,0.65,6.54,Specialized
3,6/8/20,From Warehouse Wines,2.83,0.0,31.0,0.52,5.48,Citibike
4,9/6/20,Dn ES UP WS,17.30,1.0,55.0,1.92,9.03,Specialized
...,...,...,...,...,...,...,...,...
349,9/30/19,CP BL x 2,13.60,1.0,44.0,1.73,7.85,Specialized
350,11/19/16,Down ES UP WS,16.10,1.0,56.0,1.93,8.33,Specialized
351,6/7/19,CP BL,7.47,0.0,54.0,0.90,8.30,Specialized
352,6/8/20,To Despana,3.64,0.0,35.0,0.58,6.24,Citibike


In [61]:
# Turning Index into Orig_Index Column
all_df.reset_index(inplace=True)
all_df = all_df.rename(columns = {'index': 'Orig_Index'})
all_df

,Orig_Index,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,0,11/28/16,Citibike from Work,0.81,0.0,7.0,0.12,6.94,Citibike
1,1,10/15/19,Wards Island,14.20,1.0,44.0,1.73,8.19,Specialized
2,2,7/28/19,Roosevelt Island,4.25,0.0,39.0,0.65,6.54,Specialized
3,3,6/8/20,From Warehouse Wines,2.83,0.0,31.0,0.52,5.48,Citibike
4,4,9/6/20,Dn ES UP WS,17.30,1.0,55.0,1.92,9.03,Specialized
...,...,...,...,...,...,...,...,...,...
349,349,9/30/19,CP BL x 2,13.60,1.0,44.0,1.73,7.85,Specialized
350,350,11/19/16,Down ES UP WS,16.10,1.0,56.0,1.93,8.33,Specialized
351,351,6/7/19,CP BL,7.47,0.0,54.0,0.90,8.30,Specialized
352,352,6/8/20,To Despana,3.64,0.0,35.0,0.58,6.24,Citibike


In [62]:
# Defining fields to use
columns = ['Miles', 'Duration', 'Speed','Type']
# target = ['Type']

In [63]:
# Create Dataframe with desired columns
bike_df = all_df.loc[:,columns].copy()
bike_df

,Miles,Duration,Speed,Type
0,0.81,0.12,6.94,Citibike
1,14.20,1.73,8.19,Specialized
2,4.25,0.65,6.54,Specialized
3,2.83,0.52,5.48,Citibike
4,17.30,1.92,9.03,Specialized
...,...,...,...,...
349,13.60,1.73,7.85,Specialized
350,16.10,1.93,8.33,Specialized
351,7.47,0.90,8.30,Specialized
352,3.64,0.58,6.24,Citibike


In [37]:
# Create Dataframe with desired columns
# Demonstrating different way to do it
# bike_df = all_df[['Miles', 'Duration', 'Speed','Type']]
# bike_df

In [64]:
# Splitting out Features and Target
y = bike_df["Type"]
X = bike_df.drop(columns="Type")

In [65]:
# Splitting out Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [66]:
# Checking training data shape
X_train.shape

(265, 3)

In [67]:
# Checking testing data shape
X_test.shape

(89, 3)

In [68]:
# Creating the Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

In [69]:
# Training the Model
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [70]:
# Create Predictions
y_pred = classifier.predict(X_test)

In [71]:
# Get accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9775280898876404


In [72]:
# Convert Predictions to DataFrame
pred_df = pd.DataFrame(y_pred)
pred_df.reset_index()
pred_df = pred_df.rename(columns = {0:'Prediction'})
pred_df

,Prediction
0,Specialized
1,Citibike
2,Specialized
3,Specialized
4,Specialized
...,...
84,Citibike
85,Citibike
86,Specialized
87,Citibike


In [73]:
# Convert Tests to DataFrame
test_df = pd.DataFrame(y_test)
test_df

,Type
88,Specialized
301,Citibike
333,Specialized
219,Specialized
91,Specialized
...,...
319,Citibike
234,Citibike
56,Specialized
165,Citibike


In [74]:
# Making index into a column
test_df.reset_index(inplace=True)
test_df = test_df.rename(columns = {'index': 'Orig_Index'})
test_df

,Orig_Index,Type
0,88,Specialized
1,301,Citibike
2,333,Specialized
3,219,Specialized
4,91,Specialized
...,...,...
84,319,Citibike
85,234,Citibike
86,56,Specialized
87,165,Citibike


In [75]:
# Combining the new test and predition dataframes horizontally for comparison
test_pred_df = pd.concat([test_df, pred_df], axis = 1)
test_pred_df

,Orig_Index,Type,Prediction
0,88,Specialized,Specialized
1,301,Citibike,Citibike
2,333,Specialized,Specialized
3,219,Specialized,Specialized
4,91,Specialized,Specialized
...,...,...,...
84,319,Citibike,Citibike
85,234,Citibike,Citibike
86,56,Specialized,Specialized
87,165,Citibike,Citibike


In [76]:
# Identify records where prediction is wrong
pred_errs_df = test_pred_df.loc[test_pred_df['Type'] != test_pred_df['Prediction']]
pred_errs_df

,Orig_Index,Type,Prediction
21,152,Specialized,Citibike
67,100,Specialized,Citibike


In [77]:
# Merging pred_errs_df with all_df to see details of errors
err_details_df = pd.merge(pred_errs_df, all_df, on=["Orig_Index", "Orig_Index"])
err_details_df

,Orig_Index,Type_x,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type_y
0,152,Specialized,Citibike,7/15/19,Up Down ES,3.74,0.0,33.0,0.55,6.80,Specialized
1,100,Specialized,Citibike,12/29/20,From Trek WS,3.75,0.0,26.0,0.43,8.65,Specialized


In [78]:
# Cleaning up err_details_df to eliminate redundancy and improve readability
err_details_df.drop(columns='Type_y', inplace=True)
err_details_df.rename(columns = {'Type_x' : 'Type'}, inplace=True)
err_details_df

,Orig_Index,Type,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed
0,152,Specialized,Citibike,7/15/19,Up Down ES,3.74,0.0,33.0,0.55,6.80
1,100,Specialized,Citibike,12/29/20,From Trek WS,3.75,0.0,26.0,0.43,8.65
